In [3]:
from scipy.sparse import csc_array as spcsc
from scipy import sparse
from scipy.sparse import kron as spkrn
from scipy.sparse.linalg import expm
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
import random
import numpy as np
import os
import sys
from scipy.optimize import minimize, Bounds
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import qasm_simulator
from qiskit_aer import AerSimulator
import networkx as nx
from tqdm import tqdm
from numpy import pi
from qiskit_optimization import QuadraticProgram
import matplotlib as mpl
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems import QuadraticObjective
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.algorithms import WarmStartQAOAOptimizer, CplexOptimizer
from qiskit_optimization.algorithms import SlsqpOptimizer

from docplex.mp.model import Model
import numpy as np

from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import TwoLocal
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime

algorithm_globals.random_seed = 1234


# In[26]:


import numpy as np
from itertools import product
import time
import matplotlib.pyplot as plt

In [ ]:
def create_custom_mixer(n, mixer_type='X'):
    pauli_list = []
    if mixer_type in ['X', 'Y']:
        for i in range(n):
            pauli_string = ['I'] * n
            pauli_string[i] = mixer_type
            pauli_list.append("".join(pauli_string))
    elif mixer_type == 'XY':
        for i in range(n - 1):
            pauli_list.append("".join(['I'] * i + ['X', 'X'] + ['I'] * (n - i - 2)))
            pauli_list.append("".join(['I'] * i + ['Y', 'Y'] + ['I'] * (n - i - 2)))
    else:
        raise ValueError("Invalid mixer_type. Choose from 'X', 'Y', or 'XY'.")
    return SparsePauliOp([Pauli(p) for p in pauli_list])

def create_grover_mixer(n):
    coeff = -2 / (2 ** n)
    basis_states = []
    for i in range(2 ** n):
        basis_str = format(i, f'0{n}b')
        basis_states.append(Pauli(basis_str.replace('0', 'I').replace('1', 'X')))
    identity = 'I' * n
    return SparsePauliOp(basis_states, coeffs=[coeff] * len(basis_states)) + SparsePauliOp(identity)

def create_swap_mixer(n, connectivity='full'):
    paulis = []
    coeffs = []
    def add_term(i, j, pauli_type):
        label = ['I'] * n
        label[i] = pauli_type
        label[j] = pauli_type
        paulis.append(Pauli("".join(label)))
        coeffs.append(1.0)
    if connectivity == 'full':
        pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    elif connectivity == 'ring':
        pairs = [(i, (i + 1) % n) for i in range(n)]
    for i, j in pairs:
        add_term(i, j, 'X')
        add_term(i, j, 'Y')
        add_term(i, j, 'Z')
    return SparsePauliOp(paulis, coeffs)

def brute_force_set_balancing(A):
    n = A.shape[1]
    min_val = float('inf')
    best_b_list = []

    for b_tuple in product([-1, 1], repeat=n):
        b = np.array(b_tuple)
        objective_val = np.linalg.norm(A @ b) ** 2

        if objective_val < min_val:
            min_val = objective_val
            best_b_list = [b.copy()]
        elif objective_val == min_val:
            best_b_list.append(b.copy())

    return min_val
n_values = list(range(10, 12))  # You can expand this if needed
avg_objective_brute_force = []
avg_objective_qaoa = []


# QAOA settings


mixer_names = {
    0: "X-mixer",
    1: "XY-mixer",
    2: "Full-SWAP mixer",
    3: "Ring-SWAP mixer",
    4: "Grover mixer",
    5: "Warm started"
}

n_values = list(range(10, 11))
num_trials = 1

for n in n_values:
    total_brute = 0.0
    qaoa_totals = {i: 0.0 for i in range(6)}  # Store total QAOA values for each mixer

    for _ in tqdm(range(num_trials), desc=f"Running trials for n={n}"):
        A = np.random.randint(0, 2, size=(n, n))
        Q = A.T @ A

        # --- Brute Force ---
        brute_val = brute_force_set_balancing(A)
        total_brute += brute_val

        # --- QAOA for each mixer ---
        for i in range(6):
            mdl = Model(name="SetBalancing")
            b = {j: mdl.binary_var(name=f"b_{j}") for j in range(n)}
            b_vars = {j: 2 * b[j] - 1 for j in range(n)}
            quadratic_terms = mdl.sum(Q[i, j] * b_vars[i] * b_vars[j] for i in range(n) for j in range(n))
            mdl.minimize(quadratic_terms)
            mod = from_docplex_mp(mdl)

            if i == 0:
                custom_mixer = create_custom_mixer(n, mixer_type='X')
            elif i == 1:
                custom_mixer = create_custom_mixer(n, mixer_type='XY')
            elif i == 2:
                custom_mixer = create_swap_mixer(n, connectivity='full')
            elif i == 3:
                custom_mixer = create_swap_mixer(n, connectivity='ring')
            elif i == 4:
                custom_mixer = create_grover_mixer(n)
                
            
            elif i==5:
                cobyla = COBYLA()
                cobyla.set_options(maxiter=500)
                qaoa = QAOA(sampler=Sampler(), optimizer=cobyla, reps=5)
                ws_qaoa = WarmStartQAOAOptimizer(
                    pre_solver=CplexOptimizer(),##SlsqpOptimizer()
                    relax_for_pre_solver=True,
                    qaoa=qaoa,
                    epsilon=0.01
                )
                result_ws = ws_qaoa.solve(mod)
                
            cobyla = COBYLA()
            cobyla.set_options(maxiter=500)
            qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=5, mixer=custom_mixer)
            qaoa = MinimumEigenOptimizer(qaoa_mes)
            result = qaoa.solve(mod)
            if i==5:
                qaoa_val = result_ws.fval
            else:
                qaoa_val = result.fval
            qaoa_totals[i] += qaoa_val

    # --- Print results for this n ---
    print(f"\n{'='*40}")
    print(f"n = {n}")
    print(f"Average Brute Force Objective: {total_brute / num_trials}")
    for i in range(6):
        print(f"Average QAOA Objective with {mixer_names[i]}: {qaoa_totals[i] / num_trials}")
    print(f"{'='*40}")


Running trials for n=10:   0%|                                                        | 0/1 [00:00<?, ?it/s]